In [2]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import csv

In [101]:

teamQualified = {'6':'India'}
teamResultTally = {'India':0,'Australia':0}
urls = []
results = []
for i,j in teamQualified.items():
    url = f"https://www.espncricinfo.com/team/{j}-{i}/match-schedule-fixtures-and-results"
    urls.append(url)


for u in urls:
        response = requests.get(u)
        htmlcontent = response.content
        soup = BeautifulSoup(htmlcontent,"html.parser")
        
#         print(u)
        for a in soup.find_all('div',attrs={'class':'ds-text-compact-xxs'}):
            result = soup.find_all('p',attrs = {'class':'ds-text-tight-s ds-font-regular ds-line-clamp-2 ds-text-typo-title'})

# for r in result: 
#                 print(r.text)


In [100]:
players = []
ranking = []
results = []

url = "http://www.relianceiccrankings.com/ranking/odi/batting/"

response = requests.get(url)
htmlcontent = response.content
soup = BeautifulSoup(htmlcontent,"html.parser")
        
for a in soup.find_all('tr'):
        player = soup.find_all('a',attrs={'class':'players'})
        c= 0
        for p in player:
            players.append(p.text)
            c+=1
            ranking.append(str(c))
        break

results.append(ranking)
results.append(players)

res = list(zip(*results))



In [141]:
teams = []
ranking = []
results = []

url = "https://www.icc-cricket.com/rankings/mens/team-rankings/odi"

response = requests.get(url)
htmlcontent = response.content
soup = BeautifulSoup(htmlcontent,"html.parser")
        
for a in soup.find_all('tr'):
        team = soup.find_all('span',attrs={'class':'u-hide-phablet'})
        c = 0
        for t in team:
            c+=1
            teams.append(t.text)
            ranking.append(str(c))
        break

results.append(ranking)
results.append(teams)

res = list(zip(*results))

# for r in res:
#     print(r)

In [149]:
# India, New Zealand, England, Australia, Bangladesh, Pakistan, Afghanistan

urls = []
for i in range(2018,2023):
    url = f"https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id={i};type=year"
    urls.append(url)

matches = []
winners = []


dict = {'Team':['India','Australia','New Zealand','England','Bangladesh','Pakistan','Afghanistan'],
        'Total_Matches':[0,0,0,0,0,0,0],
        'Matches_Won':[0,0,0,0,0,0,0],
        'Winning_Percentage':[0,0,0,0,0,0,0]
       }

for u in urls:

        response = requests.get(u)
        htmlcontent = response.content
        soup = BeautifulSoup(htmlcontent,"html.parser")
        
        for row in soup.findAll('table')[0].tbody.findAll('tr'):
            
            first_column = row.findAll('td')[0].contents
            for f1 in first_column: 
                matches.append(f1.text)
            second_column = row.findAll('td')[1].contents
            for f2 in second_column: 
                matches.append(f2.text)
            third_column = row.findAll('td')[2].contents
            for f3 in third_column: 
                winners.append(f3.text)
        
            


for i in matches:
    for j in range(len(dict['Team'])):
        if dict['Team'][j]==i:
            dict['Total_Matches'][j]+= 1

            
for i in winners:
    for j in range(len(dict['Team'])):
        if dict['Team'][j]==i:
            dict['Matches_Won'][j]+= 1
            
for i in range(len(dict['Team'])):
    dict['Winning_Percentage'][i] += round((dict['Matches_Won'][i]/dict['Total_Matches'][i])*100,2)
            
            
new = pd.DataFrame.from_dict(dict)
# print(new['Team'])

team = [t for t in new['Team']] #list=list(new)
rank = []
for r in range(len(res)):
    rank.append([res[r][0],res[r][1]])

new['International_Ranking'] = [1,1,1,1,1,1,1]
#print(new)
print(rank)


[['1', 'New Zealand'], ['2', 'England'], ['3', 'Australia'], ['4', 'India'], ['5', 'Pakistan'], ['6', 'South Africa'], ['7', 'Bangladesh'], ['8', 'Sri Lanka'], ['9', 'Afghanistan'], ['10', 'West Indies'], ['11', 'Ireland'], ['12', 'Scotland'], ['13', 'Zimbabwe'], ['14', 'Namibia'], ['15', 'Netherlands'], ['16', 'Oman'], ['17', 'UAE'], ['18', 'United States'], ['19', 'Nepal'], ['20', 'Papua New Guinea']]


In [150]:
new1=[]
abb=['IND','AUS','NZ','ENG','BAN','PAK','AFG']
for i in new['Team']:
    for j in range(len(rank)):
        if i==rank[j][1]:
            new1.append(rank[j][0])
print(new1)
new['International_Ranking'] = new1
new['Team_Abbreviation']=abb
print(new)

['4', '3', '1', '2', '7', '5', '9']
          Team  Total_Matches  Matches_Won  Winning_Percentage  \
0        India             87           54               62.07   
1    Australia             69           37               53.62   
2  New Zealand             57           37               64.91   
3      England             76           46               60.53   
4   Bangladesh             68           41               60.29   
5     Pakistan             61           29               47.54   
6  Afghanistan             55           27               49.09   

  International_Ranking Team_Abbreviation  
0                     4               IND  
1                     3               AUS  
2                     1                NZ  
3                     2               ENG  
4                     7               BAN  
5                     5               PAK  
6                     9               AFG  


In [151]:
new.to_csv('cricket.csv')

In [82]:
from pyspark.sql import SparkSession
#from pyspark import SparkContext, SparkConf

In [83]:
import pyspark
spark = SparkSession.builder.getOrCreate()

In [152]:
df=spark.read.csv('cricket.csv',header=True)

In [153]:
df.show()

+---+-----------+-------------+-----------+------------------+---------------------+-----------------+
|_c0|       Team|Total_Matches|Matches_Won|Winning_Percentage|International_Ranking|Team_Abbreviation|
+---+-----------+-------------+-----------+------------------+---------------------+-----------------+
|  0|      India|           87|         54|             62.07|                    4|              IND|
|  1|  Australia|           69|         37|             53.62|                    3|              AUS|
|  2|New Zealand|           57|         37|             64.91|                    1|               NZ|
|  3|    England|           76|         46|             60.53|                    2|              ENG|
|  4| Bangladesh|           68|         41|             60.29|                    7|              BAN|
|  5|   Pakistan|           61|         29|             47.54|                    5|              PAK|
|  6|Afghanistan|           55|         27|             49.09|           

In [110]:
country=[]
l3=[]
res1=[]
url = "http://www.relianceiccrankings.com/ranking/odi/batting/"
response = requests.get(url)
htmlcontent = response.content
soup = BeautifulSoup(htmlcontent,"html.parser")
for a in soup.find_all('tr'):
#     player = soup.find_all('a',attrs={'class':'players'})
    team = soup.find_all('img')
    for img in team:
        alt_tag = img.attrs['alt']    
        if alt_tag=="ICC Cricket Rankings Logo":
            continue
        country.append(alt_tag)
    break    
        
l3.append(players)
l3.append(country)

res1 = list(zip(*l3))

res=[]
for i in range(len(res1)):
    res.append([res1[i][0],res1[i][1]])
      
# print(res)


In [111]:
# res.to_csv('player_country.csv')
import csv

with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(res)


In [114]:
# column=['Player Name','Country']
# s=spark.read.csv('output.csv',header=True,column=column)

df1 = spark.read.option("inferSchema","true").csv("output.csv").toDF(
  "Player_Name","Country"
)

In [115]:
df1.show()

+--------------------+-------+
|         Player_Name|Country|
+--------------------+-------+
|          Babar Azam|    PAK|
|         Imam-ul-Haq|    PAK|
|Rassie van der Du...|     SA|
|     Quinton de Kock|     SA|
|        David Warner|    AUS|
|         Steve Smith|    AUS|
|      Jonny Bairstow|    ENG|
|         Virat Kohli|    IND|
|        Rohit Sharma|    IND|
|     Kane Williamson|     NZ|
|        Fakhar Zaman|    PAK|
|           Shai Hope|     WI|
|            Joe Root|    ENG|
|       Paul Stirling|    IRE|
|        Shreyas Iyer|    IND|
|         Tamim Iqbal|    BAN|
|          Tom Latham|     NZ|
|           Jason Roy|    ENG|
|        David Miller|     SA|
|     Mushfiqur Rahim|    BAN|
+--------------------+-------+
only showing top 20 rows



In [181]:
import pyspark.sql
df1.createOrReplaceTempView("CP")
x=spark.sql("select Country,count(Player_Name) as PC from CP group by Country")
x.createOrReplaceTempView("CP")
x.show()

+-------+---+
|Country| PC|
+-------+---+
|     SL| 10|
|    NEP|  1|
|    ZIM|  3|
|    AUS|  9|
|     WI|  5|
|    AFG|  6|
|    PNG|  1|
|     SA|  7|
|    ENG|  6|
|    SCO|  5|
|    BAN|  7|
|    NED|  3|
|    PAK|  7|
|    IRE|  3|
|    USA|  4|
|    NAM|  1|
|    UAE|  1|
|    OMA|  3|
|     NZ|  8|
|    IND| 10|
+-------+---+



In [184]:
df.createOrReplaceTempView("STATS")
d=spark.sql("select Team,Total_Matches,Matches_Won,Winning_Percentage,International_Ranking,PC from STATS , CP where STATS.Team_Abbreviation==CP.Country ")
d.show()

+-----------+-------------+-----------+------------------+---------------------+---+
|       Team|Total_Matches|Matches_Won|Winning_Percentage|International_Ranking| PC|
+-----------+-------------+-----------+------------------+---------------------+---+
|  Australia|           69|         37|             53.62|                    3|  9|
|Afghanistan|           55|         27|             49.09|                    9|  6|
|    England|           76|         46|             60.53|                    2|  6|
| Bangladesh|           68|         41|             60.29|                    7|  7|
|   Pakistan|           61|         29|             47.54|                    5|  7|
|New Zealand|           57|         37|             64.91|                    1|  8|
|      India|           87|         54|             62.07|                    4| 10|
+-----------+-------------+-----------+------------------+---------------------+---+

